In [1]:
import os
import sys
sys.path.append("../../common/src")
sys.path.append("../src")
os.chdir("../src")

In [2]:
from services.agents.db_agent import run_db_agent

INFO: [config/config.py:54 - <module>()] Namespace File not found, setting job namespace as default
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_LLAMA2CPP_LLM = False
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_TRUSS_LLAMA2 = True
INFO: [config/config.py:251 - <module>()] Can't load llm_service_models.json: 404 Secret [projects/877759600672/secrets/llm_service_password_Llama2cpp] not found or has no versions.
INFO: [config/config.py:261 - <module>()] LLM types loaded ['OpenAI-GPT3.5', 'OpenAI-GPT4', 'Cohere', 'VertexAI-Text', 'VertexAI-Chat-V1', 'VertexAI-Chat', 'Truss-Llama2-Chat']
INFO: [config/config.py:286 - <module>()] Embedding models loaded ['VertexAI-Embed

In [3]:
prompt = "Which CDT codes are medicaid approved?"

In [4]:
run_db_agent(prompt)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: CdtCode, ClaimTransaction, FqhcProvider
Thought:The tables CdtCode and ClaimTransaction seem to be the most relevant to the question. I should check their schemas to understand the structure of the data.
Action: sql_db_schema
Action Input: "CdtCode, ClaimTransaction"
Observation: 
CREATE TABLE `CdtCode` (
	`cdtCodeId` STRING, 
	`codeDescription` STRING, 
	`medicaidApproved` BOOL
)

/*
3 rows from CdtCode table:
cdtCodeId	codeDescription	medicaidApproved
D0145	oral evaluation for a patient under three years of age and counseling with primary caregiver	False
D0171	re-evaluation – post-operative office visit	False
D0190	screening of a patient	False
*/


CREATE TABLE `ClaimTransaction` (
	`claimTransactionId` INT64, 
	`fqhcProviderId` INT64, 
	`cdtCodeId` STRING, 
	`userId` INT64, 
	`userEmail` STRING, 
	`submitDateTime` TIMESTAMP
)

/*
3 rows from ClaimTransaction table:
claimTransactionId	fqh

'{\n  "columns": ["cdtCodeId", "codeDescription"],\n  "data": [\n    ["D0120", "periodic oral evaluation - established patient"],\n    ["D0140", "limited oral evaluation - problem focused"],\n    ["D0150", "comprehensive oral evaluation - new or established patient"],\n    ["D0160", "detailed and extensive oral evaluation - problem focused, by report"],\n    ["D0170", "re-evaluation - limited, problem focused (established patient; not post-operative visit)"],\n    ["D0180", "comprehensive periodontal evaluation - new or established patient"],\n    ["D0210", "intraoral - complete series of radiographic images"],\n    ["D0220", "intraoral - periapical first radiographic image"],\n    ["D0230", "intraoral - periapical each additional radiographic image"],\n    ["D0240", "intraoral - occlusal radiographic image"],\n    ["D0250", "extra-oral – 2D projection radiographic image created using a stationary radiation source, and detector"],\n    ["D0270", "bitewing - single radiographic image"],